In [ ]:

#@title  **Leiden Community Stability Analysis**

#  1: Install necessary libraries
!pip install python-igraph
!pip install leidenalg

# --- [الخطوة 0: التثبيت والاستيراد] ---
print("--- [Step 0: Installing and Importing Libraries] ---")
!git clone https://github.com/NoorBayan/Amthal.git
%cd Amthal
from utility import *
from IPython.display import display, HTML, clear_output

#  2: Your main analysis code
import pandas as pd
import networkx as nx
import leidenalg as la
import igraph as ig
from collections import Counter
import io
from itertools import combinations
import ast

# --- [الخطوة 1: تحميل وتجهيز البيانات] ---
print("--- [Step 1: Loading Data] ---")
INSTANCES_FILE_PATH = '/content/Amthal/data/processed/instances.csv'
try:
    df = pd.read_csv(INSTANCES_FILE_PATH, sep='\t', encoding='utf-16')
    df['Dominant_Concept']=[Dominant_Concept_dic[i]['en'] for i in df.Dominant_Concept]
    df['Valence']=[Valence_dic[i]['ar'] for i in df.Valence]
    df['Revelation_Phase']=[Revelation_Phase_dic[i]['ar'] for i in df.Revelation_Phase]
    df['Abstraction_Level']=[Abstraction_Level_dic[i]['ar'] for i in df.Abstraction_Level]

    print(f"Data loaded successfully. Found {len(df)} instances.\n")
except FileNotFoundError:
    print(f"⚠️ ERROR: File not found at '{INSTANCES_FILE_PATH}'. Please check the path and try again.")
    exit()

# --- [الخطوة 2: بناء الشبكة] ---
print("--- [Step 2: Building the Network] ---")
G_nx = nx.Graph()
verses = df.groupby(['Sura_No', 'Aya_No'])['Dominant_Concept'].apply(list).reset_index()

for index, row in verses.iterrows():
    concepts_in_verse = list(set(row['Dominant_Concept']))
    if len(concepts_in_verse) > 1:
        verse_text = df[(df['Sura_No'] == row['Sura_No']) & (df['Aya_No'] == row['Aya_No'])]['vers_text'].iloc[0]
        verse_length = len(str(verse_text).split())
        weight = 1 / verse_length if verse_length > 0 else 0

        for concept1, concept2 in combinations(concepts_in_verse, 2):
            if G_nx.has_edge(concept1, concept2):
                G_nx[concept1][concept2]['weight'] += weight
            else:
                G_nx.add_edge(concept1, concept2, weight=weight)
print(f"Network built. It has {G_nx.number_of_nodes()} nodes and {G_nx.number_of_edges()} edges.\n")

# --- [الخطوة 3: تحليل المجموعات (تكرار 100 مرة)] ---
print(f"--- [Step 3: Running Community Detection] ---")
G_ig = ig.Graph.from_networkx(G_nx)
num_runs = 100
all_partitions = []

print(f"Running Leiden algorithm {num_runs} times to check for stability...")
for i in range(num_runs):
    partition = la.find_partition(
        G_ig,
        la.RBConfigurationVertexPartition,
        weights='weight',
        seed=i
    )
    all_partitions.append(partition)
print("Finished all runs.\n")

# --- [الخطوة 4: تحليل استقرار النتائج] ---
print("--- [Step 4: Stability Analysis Results] ---")
partition_strings = []
for p in all_partitions:
    sorted_partition = sorted([sorted(community) for community in p])
    partition_strings.append(str(sorted_partition))

partition_counts = Counter(partition_strings)
clear_output()
print(f"Total runs: {num_runs}")
print(f"Number of unique partitions found: {len(partition_counts)}")
print("-" * 30)

if not partition_counts:
    print("No partitions were found. The network might be too small or disconnected.")
else:
    most_common_partition, count = partition_counts.most_common(1)[0]
    stability_percentage = (count / num_runs) * 100

    print(f"The most stable partition was found {count} out of {num_runs} times ({stability_percentage:.2f}% stability).")
    print("\nMost Stable Partition Details:")

    stable_partition_list = ast.literal_eval(most_common_partition)
    num_communities = len(stable_partition_list)

    # Calculate modularity for one of the stable partitions
    # First, find an index of a stable partition
    stable_partition_index = partition_strings.index(most_common_partition)
    modularity_score = all_partitions[stable_partition_index].modularity

    print(f"Number of communities found: {num_communities}")
    print(f"Modularity Score (Q): {modularity_score:.4f}")

    for i, community_indices in enumerate(stable_partition_list):
        community_nodes = [G_ig.vs[index]['_nx_name'] for index in community_indices]
        print(f"\nCommunity {i+1}:")
        print(f"  - Concepts: {', '.join(community_nodes)}")
        print(f"  - Size: {len(community_nodes)} concepts")

Total runs: 100
Number of unique partitions found: 4
------------------------------
The most stable partition was found 95 out of 100 times (95.00% stability).

Most Stable Partition Details:
Number of communities found: 4
Modularity Score (Q): -0.0031

Community 1:
  - Concepts: Path, Power, Fear, Material Properties, Cognition, Ruin, Commerce, Social Relations, Purification & Rectification
  - Size: 9 concepts

Community 2:
  - Concepts: Life, Death, Trials
  - Size: 3 concepts

Community 3:
  - Concepts: Darkness, Light, Water, Shelter, Fire
  - Size: 5 concepts

Community 4:
  - Concepts: Construction, Emotions, Transactions, Behaviors
  - Size: 4 concepts
